# SensorCaps
SensorCaps utlizes IMU sensor event or human activity characteristic feature and narration extraction. In summary, it's "sensor event captioning" or "human activity captioning." This is essential for generating context-aware question-answer pairs.

In [1]:
import os
import json
import numpy as np
from tqdm import tqdm

In [2]:
def sensor_subsampled_string(data, n=60):
    if len(data)/n>10:
        print(f"High compression: {len(data)/n}")
    indices = np.round(np.linspace(0, len(data) - 1, n)).astype(int)
    return str([list(np.round(data[idx],6)) for idx in indices])

In [3]:
from openai import OpenAI
import json


client = OpenAI()


def limubert_sample_to_sensorcaps(summary, accl_str, gyro_str):
    messages = [
        {
            "role": "system",
            "content": (
                "Please consider yourself to be an expert on gyroscope and accelerometer sensor "
                "information given as a metadata of IMU datasets."
                "You are given the IMU sensor readings of a human activity. "
                "The user also provides a brief summary of the event followed by 'Summary:'. "
                "They also give you gyroscopic and accelerometer sensor data followed by "
                "'Gyroscope:' and 'Accelerometer:' respectively. "
                "They are written in a Python list of lists format and contain x, y, and z "
                "axis data respectively. Please pay attention to the values and the signs. "
                "You should provide a comprehensive details of at least a couple of characteristic IMU "
                "features for that event would be within 10-15 words, followed by 'Features:'. "
                "The IMU features should be concise and descriptive. Separate multiple features "
                "with commas. Derive the list of features based on the given sensor data. "
                "Then, narrate the temporal event with details that are context-aware "
                "based on the sensor data, followed by 'Narration:', in a step-by-step "
                "fashion, analyzing it within 150 words or less. "
                "Although the movements might appear to be minimal, please "
                "analyze the small movements thoroughly in a logical and "
                "contextual manner, utilizing deductive thought process while "
                "being aware of the knowledge regarding the meaning of the "
                "sensor data. Don't call the movements minimal or subtle. "
                "Instead use descriptive terms that aren't vague."
            )
        },
        {
            "role": "user",
            "content": (
                f"Summary: {summary}\n"
                f"Gyroscope: {gyro_str}\n"
                f"Accelerometer: {accl_str}"
            ),
        },
    ]
    params = {
        "model": "gpt-3.5-turbo",
        "messages": messages,
        "max_tokens": 300,
    }

    result = client.chat.completions.create(**params)
    narration = result.choices[0].message.content
    messages.append({"role": "assistant", "content": narration})
    sensorcaps_sample = json.dumps({"messages": messages})
    return sensorcaps_sample

In [4]:
import random


SAMPLES_PER_DATASET = 10

root_data_dir = "/hdd/LLM/limuBERT_data/extracted_data"
datasets = sorted(os.listdir(root_data_dir))

data_file_name = "data_20_120.npy"
label_file_name = "label_20_120.npy"


with open('/hdd/LLM/limuBERT_data/dataset_activity_label.json') as json_file:
    dataset_activity_label_dict = json.load(json_file)


last_axis_dict = {
    "hhar": 2, "motion":0, "shoaib":0, "uci":0
}
with open("sensorcaps_untrained_10hz.jsonl","w") as f:
    for dataset in datasets:
        data = np.load(os.path.join(root_data_dir, dataset, data_file_name))
        label = np.load(os.path.join(root_data_dir, dataset, label_file_name))
        label_dict = dataset_activity_label_dict[dataset]
        last_axis = last_axis_dict[dataset]
        print(dataset)
        
        indices = random.sample(range(1, data.shape[0]), SAMPLES_PER_DATASET)
        for sample_index in tqdm(indices):
            sample_data = data[sample_index]
            key = str(int(label[sample_index, 0, last_axis]))
            sample_label = label_dict[str(int(label[sample_index, 0, last_axis]))]
            accl, gyro = data[0][:, 0:3], data[0][:, 3:6]
            accl, gyro = accl.tolist(), gyro.tolist()
            accl_str = sensor_subsampled_string(accl)
            gyro_str = sensor_subsampled_string(gyro)
            sensorcaps_sample = limubert_sample_to_sensorcaps(
                summary=sample_label,
                accl_str=accl_str,
                gyro_str=gyro_str
            )
            f.write(sensorcaps_sample+"\n")

hhar


  0%|                                                                                      | 0/10 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [00:39<00:00,  3.95s/it]


motion


100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [00:39<00:00,  3.90s/it]


shoaib


100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [00:46<00:00,  4.65s/it]


uci


100%|█████████████████████████████████████████████████████████████████████████████| 10/10 [00:41<00:00,  4.15s/it]


# OpenSQA

Open Sensor Question-Answering data generation: A dataset containing IMU sensor context-aware question-answer pairs that can be attributed to specific accelerometer and gyroscope data. 

In [13]:
def sensorcaps_to_opensqa(qa_context):
    messages = [
        {
            "role": "system",
            "content": (
                "Please consider yourself to be an expert on gyroscope and accelerometer sensor "
                "information of IMU datasets."
                "You are given the IMU sensor readings of a human activity. "
                "The user also provides a brief summary of the event followed by 'Summary:'. "
                "They also give you gyroscopic and accelerometer sensor data followed by "
                "'Gyroscope:' and 'Accelerometer:' respectively. "
                "Characteristic IMU features of the event are written after 'Features:'. "
                "Finally, there's a temporal narration of the events after 'Narration:'. "
                "Please generate 10 detailed question-answer pairs that require "
                "step-by-step logical deductive thinking and knowledgable analysis of the "
                "sensor data. Please make the questions complex but logical so that they "
                "require information that can be derived based on the vast knowledge about "
                "sensor data and IMU activities. The questions and answers need to "
                "acknowledge the context given by the user."
            )
        },
        {
            "role": "user",
            "content": (qa_context),
        },
    ]
    params = {
        "model": "gpt-3.5-turbo",
        "messages": messages,
        "max_tokens": 1000,
    }

    result = client.chat.completions.create(**params)
    qas = result.choices[0].message.content
    messages.append({"role": "assistant", "content": qas})
    opensqa_sample = json.dumps({"messages": messages})
    return qas, opensqa_sample

In [14]:
from ast import literal_eval


with open("sensorcaps_untrained_10hz.jsonl","r") as f:
    samples = f.readlines()
with open("opensqa_untrained_10hz.jsonl","w") as f:
    for sample in tqdm(samples):
        sample = literal_eval(sample)['messages']
        sensor_and_summary = sample[1]['content']
        feature_and_narration = sample[2]['content']
        qa_context = sensor_and_summary + "\n" + feature_and_narration
        qas, opensqa_sample = sensorcaps_to_opensqa(qa_context)
        print(qas)
        print("-----------------------------------------------")
        f.write(opensqa_sample+"\n")

  2%|█▉                                                                            | 1/40 [00:16<10:47, 16.61s/it]

1. What do the gyroscope readings close to zero on all axes indicate about the user's movement during the standing activity?
   - Answer: The gyroscope readings close to zero on all axes indicate that the user is experiencing minimal rotational movement during the standing activity.

2. How do the accelerometer values varying around -6 m/s², 1 m/s², and 8 m/s² on the x, y, and z axes respectively support the conclusion that the user is standing still?
   - Answer: The stable accelerometer values suggest that the user is not undergoing any significant acceleration or deceleration, confirming that the user is in a stationary position, standing still.

3. Can we infer any specific postural adjustments made by the user based on the subtle shifts in the accelerometer data?
   - Answer: Yes, the slight variations in the accelerometer data suggest that the user may be making subtle postural adjustments while standing, possibly indicating minor shifts in body position to maintain balance or co

  5%|███▉                                                                          | 2/40 [00:29<08:59, 14.19s/it]

1. How can we interpret the gyroscope data in this sitting activity based on the values provided?
Answer: The gyroscope data shows minimal variations around zero, indicating little to no movement. The negative values suggest slight rotations along different axes but within a confined range, typical of subtle adjustments in sitting posture.

2. What do the consistent values in the accelerometer data imply about the user's activity?
Answer: The consistent values in the accelerometer data suggest that the user is in a stable position without significant movement. This is characteristic of a sedentary posture like sitting.

3. Based on the gyroscope and accelerometer data, what kind of postural changes can be inferred during this sitting activity?
Answer: The gyroscope data indicates slight rotations and adjustments in sitting posture. The accelerometer data, remaining relatively constant, supports the inference of minor postural changes that do not involve significant movement.

4. How do

  8%|█████▊                                                                        | 3/40 [00:48<10:13, 16.59s/it]

...

1. How do the gyroscope readings support the fact that the user is climbing stairs and not engaging in another physical activity?
Answer: The gyroscope readings show variations in all axes, capturing rotational movements and orientation changes. The synchronized peaks in gyroscope data during ascending movement align with the periodic patterns in the accelerometer z-axis data, indicating repetitive motions consistent with climbing stairs, demonstrating deliberate upward movements specific to stair climbing.

2. Can we infer the user's step count based on the provided gyroscope and accelerometer data during the stair-climbing activity?
Answer: By analyzing the synchronized peaks in the accelerometer z-axis data and corresponding variations in the gyroscope data, we can potentially correlate these patterns to individual steps taken during the stair-climbing activity, providing insights into the user's step count.

3. How can we differentiate between the user climbing stairs and walking o

 98%|███████████████████████████████████████████████████████████████████████████  | 39/40 [11:02<00:16, 16.58s/it]

1. How do the gyroscope data patterns align with the characteristics of walking mentioned in the summary?
Answer: The gyroscope data exhibits consistent fluctuations around zero on all axes, reflecting the steady movement typically seen in walking. This aligns with the characteristic of a regular gait pattern associated with walking.

2. How do the accelerometer readings confirm the rhythmic nature of the walking activity described in the summary?
Answer: The accelerometer data shows repeated oscillations around 10 m/s^2 in all axes, which correspond to the impact of each step taken during walking. This cyclic variation in acceleration aligns with the rhythmic steps characteristic of walking.

3. Can the synchronized patterns displayed by both the gyroscope and accelerometer data be considered typical for the walking event described in the summary?
Answer: Yes, the synchronized patterns of the gyroscope and accelerometer data indicate a harmonious motion consistent with walking. The sm

100%|█████████████████████████████████████████████████████████████████████████████| 40/40 [11:23<00:00, 17.09s/it]

1. How can the gyroscope data, with values close to zero across all axes, confirm the user's seated posture?
- The gyroscope measures angular velocity or rotational movement. In the context of sitting, minimal movement is expected, leading to near-zero gyroscope values on all axes. Therefore, the consistent low gyroscope readings indicate the user's immobility and seated position.

2. What do the stable accelerometer readings, with small fluctuations, suggest about the user's activity during sitting?
- The accelerometer data provides information on acceleration forces experienced by the device. In a seated position, slight shifts in body posture or movements like fidgeting may cause small fluctuations in the accelerometer values. The stable readings overall, with minor variations, indicate the user's static position while sitting.

3. How do the gyroscope and accelerometer data combined support the conclusion that the user is not moving during the activity?
- The gyroscope data showing